# This is a scratch space for building functions
After functions are made, move them to the corresponding folder. 

In [1]:
from pathlib import Path
from flow_analysis_comps.PIV.PIV_process import AMF_PIV
from flow_analysis_comps.PIV.definitions import PIV_params, segmentMode
%matplotlib widget

In [2]:
data_adr = Path(r"G:\AMOLF_Data\AMOLF-SHIMIZU Dropbox\Simon van Staalduine")
plate_id = r"20250122_Plate017"
video_id = r"031"
frame_id1 = r"Img/Img_00000.tif"
frame_id2 = r"Img/Img_00001.tif"

mode = "BRIGHTFIELD"
# mode = "FLUO"

fps = 20
winsize = 40 # pixels, interrogation window size in frame A
searchsize = 50  # pixels, search area size in frame B
overlap = 30 # pixels
frame_ids = (0,3)

STN_thresh = 1.05

piv_param_obj = PIV_params(
    video_path=data_adr/video_id,
    segment_mode=segmentMode.BRIGHT,
    fps=fps,
    window_size=winsize,
    search_size=searchsize,
    overlap_size=overlap,
    stn_threshold=STN_thresh,
    px_per_mm= 1500/2
)

In [ ]:
amf_piv_obj = AMF_PIV(piv_param_obj)
amf_piv_obj.plot_raw_images(frame_ids)
amf_piv_obj.plot_segmentation()

In [ ]:
amf_piv_obj.piv_process(frame_ids, FAKE_OUTLIERS=True)
amf_piv_obj.plot_results_color()

In [ ]:
amf_piv_obj.plot_results_arrows(scale=2)